<a href="https://colab.research.google.com/github/bhgtankita/LangChain/blob/main/HR_Helper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q -U google-generativeai
!pip install pypdf2
!pip install chromadb
!pip install google.generativeai
!pip install langchain-google-genai
!pip install langchain
!pip install langchain_community
!pip install pypdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 615.5/615.5 kB 16.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 55.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 273.8/273.8 kB 22.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.7/94.7 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 73.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.2/93.2 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 97.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.8/55.8 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 159.2/159.2 kB 12.1 MB/s eta 0:0

# create_stuff_documents_chain from langchain.chains.combine_documents

**Purpose:** This function helps in creating a chain that combines information from multiple documents. It’s useful for applications where you want to synthesize or summarize content from several sources.

**Use Cases:**
Summarizing information from multiple documents into a cohesive answer.
Answering questions based on data spread across different documents.
Concatenating or “stuffing” multiple documents together into a single coherent response.

**How It Works:** This chain typically "stuffs" the documents together, enabling the language model to process and generate responses based on the combined content. It’s often used with models that have larger context windows so they can effectively handle all the combined input text.



# create_retrieval_chain from langchain.chains

**Purpose:** This function is designed to set up a retrieval-based chain, which can search a collection of documents to find the most relevant information based on a user’s query.

**Use Cases:**
Searching through large databases or knowledge bases to retrieve relevant information.
Building question-answering systems where information retrieval is a key step.
Creating chatbot interactions where the bot needs to pull specific answers from a document collection.

**How It Works:** The retrieval chain typically integrates with vector stores or other retrieval mechanisms. It searches for and retrieves the most relevant documents before processing the information to answer the query.

In [3]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain.prompts import PromptTemplate
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import CharacterTextSplitter
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain
from langchain.vectorstores import Chroma
from secret import GOOGLE_API_KEY
import os

In [4]:
os.environ["GOOGLE_API_KEY"]=GOOGLE_API_KEY

In [6]:
#Load the models
llm = ChatGoogleGenerativeAI(model="gemini-pro")
embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")

#Load the PDF and create chunks
loader = PyPDFLoader("handbook.pdf")
text_splitter = CharacterTextSplitter(
    separator=".",
    chunk_size=250,
    chunk_overlap=50,
    length_function=len,
    is_separator_regex=False,
)
pages = loader.load_and_split(text_splitter)

#Turn the chunks into embeddings and store them in Chroma
vectordb=Chroma.from_documents(pages,embeddings)

#Configure Chroma as a retriever with top_k=5
retriever = vectordb.as_retriever(search_kwargs={"k": 5})

#Create the retrieval chain
template = """
You are a helpful AI assistant.
Answer based on the context provided.
context: {context}
input: {input}
answer:
"""
prompt = PromptTemplate.from_template(template)
combine_docs_chain = create_stuff_documents_chain(llm, prompt)
retrieval_chain = create_retrieval_chain(retriever, combine_docs_chain)

#Invoke the retrieval chain
response=retrieval_chain.invoke({"input":"How do I apply for personal leave?"})

In [7]:
#Print the answer to the question
print("\n ******Result******")
print(response["answer"])


 ******Result******
Such requests shall be made in writing to t he Employer not less than fifteen (15) calendar days prior to the start thereof, and shall Employer the starting and ending dates of the requested leave
